In [ ]:
DAYS=30

In [ ]:
%run ../pathutils.ipynb
%run ../database.ipynb
%run ../export.ipynb
%run health.ipynb
%run database.ipynb
%run utils.ipynb

In [ ]:
# Each member of the list consists of:
# 1. A dew point limit
# 2. A comfort score
# 3. A human-readbale assessment of comfort level

COMFORT_MAP = [
    (5, 0, "Very Dry"),
    (10, 1, "Dry"),
    (16, 2, "Comfortable"),
    (18, 3, "Moderately Humid"),
    (21, 4, "Humid"),
    (100, 5, "Oppressive")
]

In [ ]:
import numpy as np
import pandas as pd

def dew_point_c(temperature: pd.Series, humidity: pd.Series) -> pd.Series:
    """
    Calculate dew point in degrees C given air temperature (degrees C) and relative humidity (%).
    Uses Magnus formula.
    """
    a, b = 17.27, 237.7  # constants for water over liquid range
    rh_clamped = humidity.clip(lower=1, upper=100)  # avoid log(0)
    alpha = (a * temperature / (b + temperature)) + np.log(rh_clamped / 100.0)
    return (b * alpha) / (a - alpha)

In [ ]:
def assess_comfort_level(dew_point):
    if pd.isna(dew_point):
        return -1, "Unknown"

    # Iterate over the defined levels
    for definition in COMFORT_MAP:
        # Extract this one and compare the dew point level to the value being assessed
        limit, score, label = definition
        if dew_point < limit:
            # Value is in this band so return the score and label
            return score, label


In [ ]:
# Load the readings for each sensor and produce a combined data frame
bme280_df = load_sensor_readings("bme280", DAYS)
veml7700_df = load_sensor_readings("veml7700", DAYS)
sgp40_df = load_sensor_readings("sgp40", DAYS)
combined_df = merge_sensor_readings([bme280_df, veml7700_df, sgp40_df])
combined_df.head()

In [ ]:
import numpy as np
import pandas as pd

# Calculate the dew point
combined_df["dew_point"] = dew_point_c(combined_df["temperature"], combined_df["humidity"])

# Assess the comfort level
combined_df[["comfort_score", "comfort_label"]] = pd.DataFrame(
    combined_df["dew_point"].apply(assess_comfort_level).tolist(),
    index=combined_df.index
)

# Calculate a mean comfort level per day
daily_comfort = combined_df.resample("D").agg({
    "dew_point": "mean",
    "comfort_score": "mean"
})

# Calculate mean dew-point by hour of the day
combined_df["hour"] = combined_df.index.hour
hourly_dew_point = combined_df.groupby("hour")["dew_point"].mean()
hourly_score = combined_df.groupby("hour")["comfort_score"].mean()

# Preview the data
display(daily_comfort.head())

In [ ]:
# Get the export folder path
export_folder_path = get_export_folder_path("analysis")

# Strip the timezone from the timestamp, as this will cause the export to spreadsheet to fail (Excel can't
# handle dates with timezone information)
combined_df.index = combined_df.index.tz_localize(None)

# Export the data to a spreadsheet
export_to_spreadsheet(export_folder_path, "bme280_all_mean_dew_point_comfort_score.xlsx", {
    "Data": combined_df,
    "Hourly Dew Point": hourly_dew_point
})

export_to_spreadsheet(export_folder_path, "bme280_all_diurnal_dew_point_variation.xlsx", {
    "Data": combined_df,
    "Hourly Dew Point": hourly_dew_point
})

export_to_spreadsheet(export_folder_path, "bme280_all_diurnal_comfort_score_variation.xlsx", {
    "Data": combined_df,
    "Hourly Dew Point": hourly_dew_point
})


In [ ]:
# Extract the score and label from the first comfort band and use them to construct the Y-axis label
_, min_score, min_label = COMFORT_MAP[0]
_, max_score, max_label = COMFORT_MAP[-1]
y_label = f"Score {min_score} = {min_label}, {max_score} = {max_label}"

# Daily Mean Comfort Score

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12,6))
plt.bar(daily_comfort.index, daily_comfort["comfort_score"])
plt.title("Average Daily Comfort Score")
plt.ylabel(y_label)
plt.grid(True, axis="y")
plt.ylim(min_score, max_score)

# Export to PNG or PDF, if required
export_chart(export_folder_path, "bme280_all_mean_dew_point_comfort_score", "png")

plt.tight_layout()
plt.show()

# Diurnal Dew Point Variation

In [ ]:
import matplotlib.pyplot as plt

degree_sign = u"\N{DEGREE SIGN}"

plt.figure(figsize=(12,6))

plt.plot(hourly_dew_point.index, hourly_dew_point.values)
plt.title("Average Dew Point by Hour of Day")
plt.xlabel("Hour of Day")
plt.ylabel(f"Dew point ({degree_sign}C)")
plt.grid(True)

# Export to PNG or PDF, if required
export_chart(export_folder_path, "bme280_all_diurnal_dew_point_variation", "png")

plt.tight_layout()
plt.show()

# Diurnal Comfort Score Variation

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12,6))

plt.bar(hourly_score.index, hourly_score.values)
plt.title("Average Comfort Score by Hour of Day")
plt.xlabel("Hour of Day")
plt.ylabel(y_label)
plt.grid(True)
plt.ylim(min_score, max_score)

# Export to PNG or PDF, if required
export_chart(export_folder_path, "bme280_all_diurnal_comfort_score_variation", "png")

plt.tight_layout()
plt.show()